In [1]:
import pandas as pd
import numpy as np
import ast
import plotly.express as px
import string
import nltk

nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4'],
              '.env/lib/nltk_data')


[nltk_data] Downloading package punkt to .env/lib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to .env/lib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to .env/lib/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to .env/lib/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
data = pd.read_csv('./flipkart_com-ecommerce_sample_1050.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  1050 non-null   object 
 1   crawl_timestamp          1050 non-null   object 
 2   product_url              1050 non-null   object 
 3   product_name             1050 non-null   object 
 4   product_category_tree    1050 non-null   object 
 5   pid                      1050 non-null   object 
 6   retail_price             1049 non-null   float64
 7   discounted_price         1049 non-null   float64
 8   image                    1050 non-null   object 
 9   is_FK_Advantage_product  1050 non-null   bool   
 10  description              1050 non-null   object 
 11  product_rating           1050 non-null   object 
 12  overall_rating           1050 non-null   object 
 13  brand                    712 non-null    object 
 14  product_specifications  

In [4]:
CategoryTree = data.product_category_tree.str.slice(
    start=2, stop=-2).str.split(' >> ', expand=True)
for i in CategoryTree.columns:
    CategoryTree.rename(columns={i: 'category_{}'.format(i)}, inplace=True)
CategoryTree.head(3)

,category_0,category_1,category_2,category_3,category_4,category_5,category_6
0,Home Furnishing,Curtains & Accessories,Curtains,Elegance Polyester Multicolor Abstract Eyelet ...,None,None,None
1,Baby Care,Baby Bath & Skin,Baby Bath Towels,Sathiyas Baby Bath Towels,"Sathiyas Cotton Bath Towel (3 Bath Towel, Red,...",None,None
2,Baby Care,Baby Bath & Skin,Baby Bath Towels,Eurospa Baby Bath Towels,Eurospa Cotton Terry Face Towel Set (20 PIECE ...,None,None


In [5]:
CategoryTree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category_0  1050 non-null   object
 1   category_1  1050 non-null   object
 2   category_2  1047 non-null   object
 3   category_3  679 non-null    object
 4   category_4  405 non-null    object
 5   category_5  127 non-null    object
 6   category_6  57 non-null     object
dtypes: object(7)
memory usage: 57.5+ KB


 Les 3 premières hiérarchies de catégories sont bien remplies les suivantes à moitié

 nettoyage des spécifications produits et mise sous forme de dataframe

In [6]:
# nettoyage
Specifications = data.product_specifications.str.replace(
    '"product_specification"=>', '',
    regex=True).str.replace('{"key"=>', '', regex=True).str.replace(
        ', "value"=>', ':',
        regex=True).str.replace('},', ',', regex=True).str.replace(
            '[', '', regex=True).str.replace(']}', '', regex=True)
Specifications.head(3)



0    {"Brand":"Elegance", "Designed For":"Door", "T...
1    {"Machine Washable":"Yes", "Material":"Cotton"...
2    {"Material":"Cotton Terry", "Design":"SHUVAM",...
Name: product_specifications, dtype: object

In [7]:
def try_the_eval(row):
    try:
        ast.literal_eval(row)
    except:
        print('Found bad data: {0}'.format(row))


evalError = Specifications.apply(try_the_eval)

Found bad data: {"Machine Washable":"Yes", "Brand":"Jaipur Print", "Type":"Flat", "Model Name":"jaipur117", "Material":"Cotton", "Thread Count":"140", "Model ID":"jaipur117", "Fabric Care":"machinewash, do not bleach", "Size":"King", "Color":"White", {"value"=>"waranty of the product only for manufacturing defect only and product will exchange onle when it is not used and returne its origional packing", "Flat Sheet Width":"86 inch / 220 cm", "Fitted Sheet Width":"0 cm", "Pillow Cover Width":"17 inch / 45 cm", "Pillow Cover Length":"29 inch / 75 cm", "Weight":"900 g", "Fitted Sheet Depth":"0 cm", "Fitted Sheet Length":"0 cm", "Flat Sheet Depth":"0.2 cm", "Flat Sheet Length":"104 inch / 265 cm", "Number of Contents in Sales Package":"1", "Sales Package":"1 bed sheet 2 pillow cover"}
Found bad data: {"Chronograph":"Yes", "Date Display":"Yes, Day and Date Display", "Altimeter":"No", "Barometer":"No", "Alarm Clock":"NO", "Compass":"NO", "Calendar":"No", "Luminous":"No", "Type":"Analog", "St

In [8]:
SpecificationsClean = Specifications.str.replace('{"value"=>',
                                                 '"unknown":',
                                                 regex=True)


In [9]:
evalError = SpecificationsClean.apply(try_the_eval)

Found bad data: {nil}
Found bad data: {nil}
Found bad data: {nil}
Found bad data: {"Number of Contents in Sales Package":"Pack of 3"}}
Found bad data: {"Style Code":"7007YL08"}}
Found bad data: {"Style Code":"7016YM04"}}
Found bad data: {"Style Code":"7016YM09"}}
Found bad data: {"Style Code":"7052YM07"}}
Found bad data: {"Style Code":"7926SL01"}}
Found bad data: {nil}
Found bad data: nan


In [10]:
SpecificationsClean = SpecificationsClean.str.replace('{nil}',
                                                      '{"unknown":"unknown"}',
                                                      regex=True).str.replace(
                                                          '}}',
                                                          '}',
                                                          regex=True)
SpecificationsClean.fillna('{"unknown":"unknown"}', inplace=True)

In [11]:
# mise sous forme de dictionnaire puis de dataframe
for i in SpecificationsClean.index:
    spec = ast.literal_eval(SpecificationsClean[i])
    if i == 0:
        ProdSpec = pd.DataFrame(columns=spec.keys())
        ProdSpec = ProdSpec.merge(pd.DataFrame(spec, index=[i]),
                                  on=ProdSpec.columns[ProdSpec.columns.isin(
                                      spec.keys())].to_list(),
                                  how='outer')
    else:
        ProdSpec = ProdSpec.merge(pd.DataFrame(spec, index=[i]),
                                  on=ProdSpec.columns[ProdSpec.columns.isin(
                                      spec.keys())].to_list(),
                                  how='outer')

In [12]:
ProdSpec.head()

,Brand,Designed For,Type,Model Name,Model ID,Color,Length,Number of Contents in Sales Package,Sales Package,Material,...,Error Check,Memory Configuration,Memory Clock,Technology,Re-usable,Area of Use,Other Traits,Acid Free,Transfer Paper,Printed Text
0,Elegance,Door,Eyelet,Abstract Polyester Door Curtain Set Of 2,Duster25,Multicolor,213 cm,Pack of 2,2 Curtains,Polyester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Handloomhub,Door,Eyelet,Plain With Lace,HH-014,Pink,213 cm,Pack of 2,NaN,Polyester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Handloomhub,Long Door,Eyelet,Plain With Lace,HH-036,Baby Pink,274 cm,Pack of 3,NaN,Polyester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HomeeHub,Window,Eyelet,Designer Set of 2 Window Curtains,HZCN0500541,Multicolor,152.4 cm,Pack of 2,NaN,Polyester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hargunz,Long Door,Eyelet,Crush curtains,SRVC461_9,"Brown, White",275 cm,Pack of 2,NaN,Polyester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# visualisation des colonnes comportant le plus de données
ProdSpec.isna().sum().sort_values().head(15)

Type                                   312
Brand                                  409
Color                                  472
Ideal For                              511
Sales Package                          527
unknown                                622
Material                               626
Model Name                             631
Width                                  636
Number of Contents in Sales Package    684
Weight                                 699
Height                                 733
Style Code                             758
Model Number                           761
Model ID                               786
dtype: int64

In [14]:
# conservation des 7 colonnes contenant le plus de données
ProdSpecClean = ProdSpec[ProdSpec.isna().sum().sort_values().head(
    8).index.drop('unknown').to_list()]
ProdSpecCleanFill = ProdSpecClean.fillna('unknown')
ProdSpecCleanFill.head()

,Type,Brand,Color,Ideal For,Sales Package,Material,Model Name
0,Eyelet,Elegance,Multicolor,unknown,2 Curtains,Polyester,Abstract Polyester Door Curtain Set Of 2
1,Eyelet,Handloomhub,Pink,unknown,unknown,Polyester,Plain With Lace
2,Eyelet,Handloomhub,Baby Pink,unknown,unknown,Polyester,Plain With Lace
3,Eyelet,HomeeHub,Multicolor,unknown,unknown,Polyester,Designer Set of 2 Window Curtains
4,Eyelet,Hargunz,"Brown, White",unknown,unknown,Polyester,Crush curtains


In [15]:
# utilisation des 3 premières branches de catégories
TextData = CategoryTree.iloc[:, :3].merge(
    ProdSpecCleanFill, left_index=True,
    right_index=True).merge(data[['pid', 'description']],
                            left_index=True,
                            right_index=True).set_index('pid').reset_index()

In [16]:
# liste des identifiants produits et des fichiers d'image associés
ImgList = data[['pid', 'image']]

In [17]:
# tokenisation
Tokens = {}
for r in range(len(TextData)):
    Tokens[TextData.pid[r]] = nltk.word_tokenize(
        TextData.loc[r, 'description'].lower())

In [18]:
# nettoyage stopwords, ponctuation et nombres
stopW = nltk.corpus.stopwords.words('english')
for p in string.punctuation:
    stopW.append(p)
TokensClean = {}
for r in range(len(TextData)):
    TokensClean[TextData.pid[r]] = [
        word for word in Tokens[TextData.pid[r]] if not word.isdigit()
        if word not in stopW
    ]


In [19]:
# lemmatisation
Lems = {}
for r in range(len(TextData)):
    Lems[TextData.pid[r]] = [
        nltk.WordNetLemmatizer().lemmatize(word)
        for word in TokensClean[TextData.pid[r]]
    ]